In [40]:
import numpy as np
import pandas as pd
import librosa
import os
import matplotlib.pyplot as plt
import IPython.display as ipd
import warnings

In [3]:
data_set=pd.read_csv("../input/hindi-speech-classification/dataset/train.csv")
data_set.head(10)

In [44]:
data_set.isna().sum()

In [39]:
filename=data_set["file_id"][1]
file_name = os.path.join(os.path.abspath('../input/hindi-speech-classification/dataset/train')+'/'+str(filename)+'.mp3')
warnings.simplefilter("ignore")
audio_data,sample_rate=librosa.load(file_name)
file_name

In [46]:

filename=data_set["file_id"][3]
file_name = os.path.join(('../input/hindi-speech-classification/dataset/train/')+filename+'.mp3')
audio_data,sample_rate=librosa.load(file_name)
warnings.simplefilter("ignore")
plt.figure(figsize=(15,10))
plt.plot(audio_data)
ipd.Audio(file_name)

In [29]:
mfccs=librosa.feature.mfcc(y=audio_data,sr=sample_rate,n_mfcc=50)

In [30]:
mfccs=np.mean(mfccs.T,axis=0)

In [32]:
mfccs

In [34]:
extract=[]
extract.append(mfccs)

In [35]:
extract=np.array(extract)

In [36]:
extract.shape

## Freature Extraction

In [48]:
def feature_extractor(file_name):
    warnings.simplefilter("ignore")
    audio_data,sample_rate=librosa.load(file_name)
    mfccs=librosa.feature.mfcc(y=audio_data,sr=sample_rate,n_mfcc=50)
    mfccs=np.mean(mfccs.T,axis=0)
    return mfccs

In [49]:
extracted_data=[]
labels=[]

for i in range(len(data_set)):
    filename=data_set["file_id"][i]
    file_name = os.path.join(('../input/hindi-speech-classification/dataset/train/')+filename+'.mp3')
    labels.append(data_set["gender"][i])
    data=feature_extractor(file_name)
    extracted_data.append(data)

In [51]:
new_data=np.array(extracted_data)

In [52]:
new_data.shape

In [53]:
from sklearn.preprocessing import LabelEncoder
encoder=LabelEncoder()
new_labels=encoder.fit_transform(labels)

In [55]:
new_labels

In [60]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(new_data,new_labels,test_size=0.2,random_state=42)

In [62]:
from keras.models import Sequential
from keras.layers import Dense,Dropout

In [64]:
model=Sequential()
model.add(Dense(units=200,input_shape=(50,)))
model.add(Dropout(0.5))

model.add(Dense(units=128))
model.add(Dropout(0.5))

model.add(Dense(units=100))
model.add(Dropout(0.5))

model.add(Dense(units=1,activation="sigmoid"))

In [66]:
model.compile(optimizer="adam",loss="binary_crossentropy",metrics=["accuracy"])
model.fit(x_train,y_train,validation_data=(x_test,y_test),epochs=100)

## Testing Data

In [89]:
test_set=pd.read_csv("../input/hindi-speech-classification/dataset/test.csv")
test_data=[]
test_labels=[]

for i in range(len(test_set)):
    filename=test_set["file_id"][i]
    file_name=os.path.join("../input/hindi-speech-classification/dataset/test/"+filename+".mp3")
    test_labels.append(test_set["gender"][i])
    data=feature_extractor(file_name)
    test_data.append(data)
    

In [87]:
test_set.head(4)

In [92]:
new_test_data=np.array(test_data)
new_test_labels=encoder.transform(test_labels)

## Testing Data Prediction

In [95]:
predict=model.predict(new_test_data)

In [97]:
new_pred=(predict>0.5)

In [102]:
new_pred=np.multiply(new_pred,1)

In [104]:
new_pred.shape

In [106]:
new_test_labels=new_test_labels.reshape(len(new_test_labels),1)

In [107]:
new_test_labels.shape

In [112]:
from sklearn.metrics import accuracy_score,confusion_matrix
accuracy_score(new_test_labels,new_pred)*100

In [113]:
confusion_matrix(new_test_labels,new_pred)

## Train Data Prediction

In [125]:
train_predict=model.predict(new_data)

In [126]:
train_predict=(train_predict>0.5)
train_predict=np.multiply(train_predict,1)

In [127]:
new_labels=new_labels.reshape(len(new_labels),1)

In [128]:
new_labels.shape

In [130]:
accuracy_score(new_labels,train_predict)*100

In [131]:
confusion_matrix(new_labels,train_predict)

In [132]:
model.save("hindi_speech_prediction_model.h5")

In [ ]:
import librosa
audio,sample_rate=librosa.load()